In [1]:
import tensorflow as tf
import pandas as pd
import keras
train = pd.read_csv("dataset_train_set.csv")
test = pd.read_csv("dataset_dev_set.csv")

Using TensorFlow backend.


In [2]:
X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [3]:
X_train.drop(['aPosLS','hPosLS'],inplace=True,axis=1)
X_test.drop(['aPosLS','hPosLS'],inplace=True,axis=1)

In [4]:
X_train.head()

,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,aPos,aGSPG,aGCPG,aCSPG,aPtsPG,...,aPTSPG7,aGD3,aGD5,aGD7,hSpent,hIncome,hNet,aSpent,aIncome,aNet
0,4.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18.72,0.479,-18.241,18.07,1.320,-16.750
1,5.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,47.15,34.830,-12.320,8.92,38.160,29.240
2,6.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17.10,23.650,6.550,29.52,9.000,-20.520
3,7.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.99,6.260,-5.730,0.00,0.131,0.131
4,10.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,47.84,10.920,-36.920,31.03,32.000,0.970


In [5]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[1000]

(4180, 34)


array([ 0.03925603,  0.01678195,  0.00942145,  0.00412188,  0.01678195,
        0.11776808,  0.01266007,  0.01187495,  0.00323862,  0.01266007,
        0.1222629 ,  0.06420323,  0.10631514,  0.05193572,  0.00081456,
        0.0009814 ,  0.00981401,  0.00785121,  0.01121741,  0.00326806,
       -0.0019628 ,  0.0014034 ,  0.01308207,  0.01962801,  0.01962801,
        0.00326806,  0.00785121,  0.00981401,  0.        ,  0.01854847,
        0.01854847,  0.03974673,  0.7075899 ,  0.66784317])

In [6]:
y_train.value_counts()

2    1971
0    1120
1    1089
Name: target, dtype: int64

In [7]:
whome = 1
wdraw = 1971/1089
waway = 1971/1120
print("{},{},{}".format(whome,wdraw,waway))

1,1.8099173553719008,1.7598214285714286


In [8]:
class_weights = {0:1.7598,
                 1:1.8099,
                 2:1}

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
model.fit(X_train_norm, y_train.values, epochs=15, validation_data=(X_test_norm,y_test), class_weight=class_weights)

Train on 4180 samples, validate on 1520 samples
Epoch 1/15
4180/4180 [==============================] - 1s 176us/step - loss: 1.0879 - acc: 0.3751 - val_loss: 1.0633 - val_acc: 0.4803
Epoch 2/15
4180/4180 [==============================] - 0s 60us/step - loss: 1.0775 - acc: 0.4409 - val_loss: 1.0582 - val_acc: 0.4836
Epoch 3/15
4180/4180 [==============================] - 0s 62us/step - loss: 1.0701 - acc: 0.4462 - val_loss: 1.0458 - val_acc: 0.4921
Epoch 4/15
4180/4180 [==============================] - 0s 89us/step - loss: 1.0632 - acc: 0.4462 - val_loss: 1.0336 - val_acc: 0.4875
Epoch 5/15
4180/4180 [==============================] - 0s 55us/step - loss: 1.0587 - acc: 0.4548 - val_loss: 1.0312 - val_acc: 0.4921
Epoch 6/15
4180/4180 [==============================] - 0s 54us/step - loss: 1.0554 - acc: 0.4603 - val_loss: 1.0313 - val_acc: 0.4928
Epoch 7/15
4180/4180 [==============================] - 0s 54us/step - loss: 1.0523 - acc: 0.4593 - val_loss: 1.0326 - val_acc: 0.4895
Epoch 

In [12]:
val_loss, val_acc = model.evaluate(X_test_norm, y_test)
print(val_loss, val_acc)

1520/1520 [==============================] - 0s 29us/step
1.0217756472135846 0.48947368421052634


In [13]:
predictions = model.predict_classes(X_test_norm)
preds = list(predictions)
pd.Series(list(predictions)).value_counts()

0    740
2    703
1     77
dtype: int64

In [14]:
predictions[0]

2

In [15]:
correct_pred_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred == real:
        correct_pred_dict[pred] += 1
correct_pred_dict

{0: 308, 1: 20, 2: 416}

In [16]:
outcomes_incorrectly_caught_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred != real:
        outcomes_incorrectly_caught_dict[real] += 1
outcomes_incorrectly_caught_dict

{0: 152, 1: 352, 2: 272}